In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [7]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
niter = 500
rank = 80
tau = 3
gamma = 0.5
rho = 0.5
alpha = 1



print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
    d0 = 561 #561 =3*11*17

    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 512
    d6 = 6 


    W1 = 0.2*init.kaiming_normal_(torch.empty(d1, d0, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = 0.2*init.kaiming_normal_(torch.empty(d2, d1, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = 0.2*init.kaiming_normal_(torch.empty(d3, d2, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = 0.2*init.kaiming_normal_(torch.empty(d4, d3, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)

    W5 = 0.2*init.kaiming_normal_(torch.empty(d5, d4, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())
    factors5 = tensor_train(W5_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W5_tl_tensor_rec = tt_to_tensor(factors5)
    b5 = 0*torch.ones(d5, 1, device=device)


    W6 = 0.2*init.kaiming_normal_(torch.empty(d6, d5, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    b6 = 0*torch.ones(d6, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = nn.ReLU()(U5)
    U6 = torch.addmm(b6.repeat(1, N), W6, V5)
    V6 = U6 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V6 = (y_one_hot + gamma*U6 + alpha*V6)/(1 + gamma + alpha)

        # update U4 
        U6 = (gamma*V6 + rho*(torch.mm(W6,V5) + b6.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W6, b6 = updateWb_org(U6,V5,W6,b6,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)


        # update for 5th layer
        # update V3
        V5 = updateV(U5,U6,W6,b6,rho,gamma)

        # update U3
        U5 = relu_prox(V5,(rho*torch.addmm(b5.repeat(1,N), W5, V4) + alpha*U5)/(rho + alpha),(rho + alpha)/gamma,d5,N)

        # update W3 and b3
        W5, b5 = updateWb(U5,V4,W5,b5,W5_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4))
        W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors5 = tensor_train(W5_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W5_tl_tensor_rec = tt_to_tensor(factors5)


  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        a5_train = nn.ReLU()(torch.addmm(b5.repeat(1, N), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b6.repeat(1, N), W6, a5_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        a5_test = nn.ReLU()(torch.addmm(b5.repeat(1, N_test), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_test))
        pred_test = torch.argmax(torch.addmm(b6.repeat(1, N_test), W6, a5_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V6,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b6.repeat(1,N), W6, V5),U6,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,nn.ReLU()(U5),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V6,U6,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W5.reshape((4,4,4,4,4,4,4,4,4)),torch.as_tensor(W5_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))
    factors5_shape=[f.shape for f in factors5]
    Sum_of_variables_factors5=sum(list(x*y*z for x,y,z in factors5_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4+Sum_of_variables_factors5

    CR_1=((total_variabels)+(d5*d6))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5+d5*d6)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#     #this postion to add new row into existing table
#         df=pd.read_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
#         new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
#                    'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
#                    'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1],'seed':seed} 
#         df=df.append(new_row,ignore_index=True)
#         df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv',index=False)


filename= "KaimingNormal_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/5 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 3 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 1.747732400894165 - sq_loss: 661.7024536132812 - tot_loss: 1120.028665292557 - acc: 0.19518498367791076 - val_acc: 0.19002375296912113
Repeatition 1 Epoch 2 / 500 
 - time: 1.5102794170379639 - sq_loss: 294.08990478515625 - tot_loss: 516.7308813240379 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 3 / 500 
 - time: 1.4997429847717285 - sq_loss: 161.03677368164062 - tot_loss: 287.20736234635115 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 4 / 500 
 - time: 1.4970824718475342 - sq_loss: 87.26002502441406 - tot_loss: 168.03364862222224 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 5 / 500 
 - time: 1.5029571056365967 - sq_loss: 46.950496673583984 - tot_loss: 103.16803779546171 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 6 / 500 
 - time:

Repeatition 1 Epoch 48 / 500 
 - time: 1.483574628829956 - sq_loss: 0.0004062313528265804 - tot_loss: 2.620841612617369 - acc: 0.7962459194776932 - val_acc: 0.7631489650492026
Repeatition 1 Epoch 49 / 500 
 - time: 1.484426498413086 - sq_loss: 0.0003792922361753881 - tot_loss: 2.522106731961685 - acc: 0.7993743199129488 - val_acc: 0.7682388870037327
Repeatition 1 Epoch 50 / 500 
 - time: 1.4953410625457764 - sq_loss: 0.0003544948121998459 - tot_loss: 2.4947021558400593 - acc: 0.8037268770402611 - val_acc: 0.7733288089582626
Repeatition 1 Epoch 51 / 500 
 - time: 1.4986674785614014 - sq_loss: 0.0003315550275146961 - tot_loss: 2.3950337015303376 - acc: 0.8064472252448314 - val_acc: 0.7760434340006787
Repeatition 1 Epoch 52 / 500 
 - time: 1.488668441772461 - sq_loss: 0.00031020824098959565 - tot_loss: 2.3085467228793277 - acc: 0.80930359085963 - val_acc: 0.7790973871733967
Repeatition 1 Epoch 53 / 500 
 - time: 1.5009796619415283 - sq_loss: 0.00029044950497336686 - tot_loss: 2.2279534288

Repeatition 1 Epoch 95 / 500 
 - time: 1.5013854503631592 - sq_loss: 2.6474568585399538e-05 - tot_loss: 0.8641120160175433 - acc: 0.889145810663765 - val_acc: 0.8598574821852731
Repeatition 1 Epoch 96 / 500 
 - time: 1.4946446418762207 - sq_loss: 2.5199746232829057e-05 - tot_loss: 0.8072599687750426 - acc: 0.889961915125136 - val_acc: 0.8605361384458772
Repeatition 1 Epoch 97 / 500 
 - time: 1.5004525184631348 - sq_loss: 2.4036740796873346e-05 - tot_loss: 0.8003528210815603 - acc: 0.8915941240478781 - val_acc: 0.8618934509670851
Repeatition 1 Epoch 98 / 500 
 - time: 1.5190937519073486 - sq_loss: 2.2960601199883968e-05 - tot_loss: 0.8027522411164227 - acc: 0.8924102285092492 - val_acc: 0.8612147947064812
Repeatition 1 Epoch 99 / 500 
 - time: 1.4902091026306152 - sq_loss: 2.189930273743812e-05 - tot_loss: 0.7845368557500478 - acc: 0.8932263329706203 - val_acc: 0.8632507634882932
Repeatition 1 Epoch 100 / 500 
 - time: 1.4996137619018555 - sq_loss: 2.0883426259388216e-05 - tot_loss: 0.7

Repeatition 1 Epoch 141 / 500 
 - time: 1.501483678817749 - sq_loss: 5.473082637763582e-06 - tot_loss: 0.5101880476490948 - acc: 0.9289989118607181 - val_acc: 0.9022734984730234
Repeatition 1 Epoch 142 / 500 
 - time: 1.5095438957214355 - sq_loss: 5.362458068702836e-06 - tot_loss: 0.4677324719718854 - acc: 0.9298150163220892 - val_acc: 0.9032914828639295
Repeatition 1 Epoch 143 / 500 
 - time: 1.503016471862793 - sq_loss: 5.247424269327894e-06 - tot_loss: 0.4792457077082304 - acc: 0.9302230685527747 - val_acc: 0.9043094672548354
Repeatition 1 Epoch 144 / 500 
 - time: 1.51513671875 - sq_loss: 5.147996489540674e-06 - tot_loss: 0.4792513644929528 - acc: 0.9311751904243744 - val_acc: 0.9043094672548354
Repeatition 1 Epoch 145 / 500 
 - time: 1.5258362293243408 - sq_loss: 5.048670573160052e-06 - tot_loss: 0.45714805869121733 - acc: 0.9315832426550599 - val_acc: 0.9056667797760435
Repeatition 1 Epoch 146 / 500 
 - time: 1.511655569076538 - sq_loss: 4.961523245583521e-06 - tot_loss: 0.459036

Repeatition 1 Epoch 187 / 500 
 - time: 1.5078575611114502 - sq_loss: 3.288208290541661e-06 - tot_loss: 0.3740496403875966 - acc: 0.9462731229597389 - val_acc: 0.9270444519850696
Repeatition 1 Epoch 188 / 500 
 - time: 1.5162296295166016 - sq_loss: 3.276325969636673e-06 - tot_loss: 0.3998713019700908 - acc: 0.9468171926006529 - val_acc: 0.9270444519850696
Repeatition 1 Epoch 189 / 500 
 - time: 1.4892592430114746 - sq_loss: 3.255600631746347e-06 - tot_loss: 0.36013551065693505 - acc: 0.9473612622415669 - val_acc: 0.9280624363759755
Repeatition 1 Epoch 190 / 500 
 - time: 1.5047457218170166 - sq_loss: 3.233539928260143e-06 - tot_loss: 0.38233223004454686 - acc: 0.9476332970620239 - val_acc: 0.9273837801153716
Repeatition 1 Epoch 191 / 500 
 - time: 1.5144708156585693 - sq_loss: 3.2129653391166357e-06 - tot_loss: 0.37705162790193114 - acc: 0.9483133841131665 - val_acc: 0.9294197488971836
Repeatition 1 Epoch 192 / 500 
 - time: 1.5100979804992676 - sq_loss: 3.1994552500691498e-06 - tot_lo

...

Repeatition 1 Epoch 279 / 500 
 - time: 1.5164015293121338 - sq_loss: 2.359212885494344e-06 - tot_loss: 0.32870431488199614 - acc: 0.9609630032644179 - val_acc: 0.9433322022395657
Repeatition 1 Epoch 280 / 500 
 - time: 1.501964807510376 - sq_loss: 2.355447122681653e-06 - tot_loss: 0.3014856093911149 - acc: 0.9606909684439608 - val_acc: 0.9429928741092637
Repeatition 1 Epoch 281 / 500 
 - time: 1.5056579113006592 - sq_loss: 2.350493559788447e-06 - tot_loss: 0.3279577453233333 - acc: 0.9608269858541894 - val_acc: 0.9429928741092637
Repeatition 1 Epoch 282 / 500 
 - time: 1.4887018203735352 - sq_loss: 2.3434888589690672e-06 - tot_loss: 0.33993214224085655 - acc: 0.9613710554951034 - val_acc: 0.9426535459789617
Repeatition 1 Epoch 283 / 500 
 - time: 1.4862923622131348 - sq_loss: 2.3374286683974788e-06 - tot_loss: 0.34198278552980454 - acc: 0.9615070729053319 - val_acc: 0.9426535459789617
Repeatition 1 Epoch 284 / 500 
 - time: 1.4796874523162842 - sq_loss: 2.3287600470212055e-06 - tot_lo

Repeatition 1 Epoch 325 / 500 
 - time: 1.5148100852966309 - sq_loss: 2.111140020133462e-06 - tot_loss: 0.40525744931006 - acc: 0.9639553862894451 - val_acc: 0.9450288428910757
Repeatition 1 Epoch 326 / 500 
 - time: 1.524599313735962 - sq_loss: 2.1075916265544947e-06 - tot_loss: 0.3331333565344341 - acc: 0.9640914036996736 - val_acc: 0.9453681710213777
Repeatition 1 Epoch 327 / 500 
 - time: 1.5118377208709717 - sq_loss: 2.10231314667908e-06 - tot_loss: 0.3288837512563916 - acc: 0.9640914036996736 - val_acc: 0.9453681710213777
Repeatition 1 Epoch 328 / 500 
 - time: 1.514941692352295 - sq_loss: 2.0969391698599793e-06 - tot_loss: 0.317444891142852 - acc: 0.9640914036996736 - val_acc: 0.9453681710213777
Repeatition 1 Epoch 329 / 500 
 - time: 1.4981038570404053 - sq_loss: 2.0942779883625917e-06 - tot_loss: 0.2996512735296939 - acc: 0.9642274211099021 - val_acc: 0.9453681710213777
Repeatition 1 Epoch 330 / 500 
 - time: 1.4960808753967285 - sq_loss: 2.087925849991734e-06 - tot_loss: 0.31

Repeatition 1 Epoch 371 / 500 
 - time: 1.4946808815002441 - sq_loss: 1.9127141968056094e-06 - tot_loss: 0.33857715344301287 - acc: 0.9674918389553863 - val_acc: 0.9480827960637936
Repeatition 1 Epoch 372 / 500 
 - time: 1.5009996891021729 - sq_loss: 1.9082763174083084e-06 - tot_loss: 0.34700187092092794 - acc: 0.9669477693144722 - val_acc: 0.9480827960637936
Repeatition 1 Epoch 373 / 500 
 - time: 1.5079121589660645 - sq_loss: 1.9039842982238042e-06 - tot_loss: 0.2829070221640011 - acc: 0.9676278563656148 - val_acc: 0.9480827960637936
Repeatition 1 Epoch 374 / 500 
 - time: 1.514195442199707 - sq_loss: 1.8997869801751222e-06 - tot_loss: 0.3026107529937647 - acc: 0.9676278563656148 - val_acc: 0.9480827960637936
Repeatition 1 Epoch 375 / 500 
 - time: 1.5044610500335693 - sq_loss: 1.8976089677380514e-06 - tot_loss: 0.3155206507150927 - acc: 0.9673558215451578 - val_acc: 0.9480827960637936
Repeatition 1 Epoch 376 / 500 
 - time: 1.4974758625030518 - sq_loss: 1.8955291807287722e-06 - tot_

Repeatition 1 Epoch 417 / 500 
 - time: 1.5123319625854492 - sq_loss: 1.7551220707900939e-06 - tot_loss: 0.32382453224920926 - acc: 0.9704842219804135 - val_acc: 0.9494401085850017
Repeatition 1 Epoch 418 / 500 
 - time: 1.512721061706543 - sq_loss: 1.7510916450191871e-06 - tot_loss: 0.3201326106953468 - acc: 0.970076169749728 - val_acc: 0.9497794367153037
Repeatition 1 Epoch 419 / 500 
 - time: 1.5157065391540527 - sq_loss: 1.747920578054618e-06 - tot_loss: 0.318557154312864 - acc: 0.9702121871599565 - val_acc: 0.9494401085850017
Repeatition 1 Epoch 420 / 500 
 - time: 1.516000747680664 - sq_loss: 1.7445431694795843e-06 - tot_loss: 0.31212945877480447 - acc: 0.970348204570185 - val_acc: 0.9501187648456056
Repeatition 1 Epoch 421 / 500 
 - time: 1.5170247554779053 - sq_loss: 1.7410778809789917e-06 - tot_loss: 0.3234533375574813 - acc: 0.970348204570185 - val_acc: 0.9501187648456056
Repeatition 1 Epoch 422 / 500 
 - time: 1.5077383518218994 - sq_loss: 1.7383588328812039e-06 - tot_loss: 

Repeatition 1 Epoch 463 / 500 
 - time: 1.518796682357788 - sq_loss: 1.616961299077957e-06 - tot_loss: 0.3363725191018778 - acc: 0.9732045701849836 - val_acc: 0.9514760773668137
Repeatition 1 Epoch 464 / 500 
 - time: 1.5393218994140625 - sq_loss: 1.6149010662047658e-06 - tot_loss: 0.33301120991203703 - acc: 0.9736126224156693 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 465 / 500 
 - time: 1.520601749420166 - sq_loss: 1.6121723547257716e-06 - tot_loss: 0.3158680360960071 - acc: 0.9730685527747551 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 466 / 500 
 - time: 1.528000831604004 - sq_loss: 1.6107549072330585e-06 - tot_loss: 0.33596893381235216 - acc: 0.9730685527747551 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 467 / 500 
 - time: 1.5100915431976318 - sq_loss: 1.6087697076727636e-06 - tot_loss: 0.3356267255134471 - acc: 0.9719804134929271 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 468 / 500 
 - time: 1.5011541843414307 - sq_loss: 1.6068247532530222e-06 - tot_los

In [8]:
seed

10